In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np

transform = transforms.Compose([
    transforms.ToTensor()
])

train_dataset = torchvision.datasets.CIFAR10(root='./data/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.CIFAR10(root='./data/', train=False, transform=transform, download=True)

train_images = []
train_labels = []
for img, label in train_dataset:
    train_images.append(img.numpy())
    train_labels.append(label)

test_images = []
test_labels = []
for img, label in test_dataset:
    test_images.append(img.numpy())
    test_labels.append(label)

train_images = np.array(train_images)
train_labels = np.array(train_labels)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

Files already downloaded and verified
Files already downloaded and verified
(50000, 3, 32, 32)
(50000,)
(10000, 3, 32, 32)
(10000,)


In [3]:
pip install opencv-python


Note: you may need to restart the kernel to use updated packages.


In [4]:
import cv2

def conv2d_forward(X, W, b, stride=1, padding=0):
    N, C, H, W_in = X.shape
    F, _, HH, WW = W.shape
    out_h = (H + 2*padding - HH)//stride + 1
    out_w = (W_in + 2*padding - WW)//stride + 1

    X_pad = np.pad(X, ((0,0),(0,0),(padding,padding),(padding,padding)), mode='constant')
    out = np.zeros((N, F, out_h, out_w))

    for n in range(N):
        for f in range(F):
            for i in range(out_h):
                for j in range(out_w):
                    h_start = i * stride
                    w_start = j * stride
                    window = X_pad[n, :, h_start:h_start+HH, w_start:w_start+WW]
                    out[n, f, i, j] = np.sum(window * W[f]) + b[f]
    return out

def relu_forward(X):
    return np.maximum(0, X)

def maxpool_forward(X, size=2, stride=2):
    N, C, H, W_in = X.shape
    out_h = (H - size)//stride + 1
    out_w = (W_in - size)//stride + 1
    out = np.zeros((N, C, out_h, out_w))

    for n in range(N):
        for c in range(C):
            for i in range(out_h):
                for j in range(out_w):
                    h_start = i * stride
                    w_start = j * stride
                    window = X[n, c, h_start:h_start+size, w_start:w_start+size]
                    out[n, c, i, j] = np.max(window)
    return out

def fc_forward(X, W, b):
    return X @ W + b

def softmax_crossentropy_loss(logits, labels):
    shifted_logits = logits - np.max(logits, axis=1, keepdims=True)
    exp_logits = np.exp(shifted_logits)
    probs = exp_logits / np.sum(exp_logits, axis=1, keepdims=True)

    N = logits.shape[0]
    loss = -np.sum(np.log(probs[np.arange(N), labels])) / N

    dlogits = probs.copy()
    dlogits[np.arange(N), labels] -= 1
    dlogits /= N

    return loss, dlogits



In [5]:
def alexnet_forward(X, params):
    out = conv2d_forward(X, params['W1'], params['b1'], stride=1, padding=1)
    out = relu_forward(out)
    out = maxpool_forward(out, size=2, stride=2)

    out = conv2d_forward(out, params['W2'], params['b2'], stride=1, padding=2)
    out = relu_forward(out)
    out = maxpool_forward(out, size=2, stride=2)

    out = conv2d_forward(out, params['W3'], params['b3'], stride=1, padding=1)
    out = relu_forward(out)

    out = conv2d_forward(out, params['W4'], params['b4'], stride=1, padding=1)
    out = relu_forward(out)

    out = conv2d_forward(out, params['W5'], params['b5'], stride=1, padding=1)
    out = relu_forward(out)
    out = maxpool_forward(out, size=2, stride=2)

    N = out.shape[0]
    out = out.reshape(N, -1)

    out = fc_forward(out, params['W6'], params['b6'])
    out = relu_forward(out)

    out = fc_forward(out, params['W7'], params['b7'])
    out = relu_forward(out)

    out = fc_forward(out, params['W8'], params['b8'])

    return out


In [6]:
def initialize_params():
    params = {}
    params['W1'] = np.random.randn(64, 3, 3, 3) * np.sqrt(2. / (3*3*3))
    params['b1'] = np.zeros(64)
    params['W2'] = np.random.randn(192, 64, 5, 5) * np.sqrt(2. / (64*5*5))
    params['b2'] = np.zeros(192)
    params['W3'] = np.random.randn(384, 192, 3, 3) * np.sqrt(2. / (192*3*3))
    params['b3'] = np.zeros(384)
    params['W4'] = np.random.randn(256, 384, 3, 3) * np.sqrt(2. / (384*3*3))
    params['b4'] = np.zeros(256)
    params['W5'] = np.random.randn(256, 256, 3, 3) * np.sqrt(2. / (256*3*3))
    params['b5'] = np.zeros(256)
    params['W6'] = np.random.randn(256*4*4, 4096) * np.sqrt(2. / (256*4*4))
    params['b6'] = np.zeros(4096)
    params['W7'] = np.random.randn(4096, 4096) * np.sqrt(2. / 4096)
    params['b7'] = np.zeros(4096)
    params['W8'] = np.random.randn(4096, 10) * np.sqrt(2. / 4096)
    params['b8'] = np.zeros(10)
    return params


In [ ]:
params = initialize_params()

batch_X = train_images[:32]
batch_y = train_labels[:32]

logits = alexnet_forward(batch_X, params)

loss, probs = softmax_crossentropy_loss(logits, batch_y)

print(f"loss = {loss:.4f}")


In [7]:
def fc_backward(dout, X, W, b):
    dX = dout @ W.T
    dW = X.T @ dout
    db = np.sum(dout, axis=0)
    return dX, dW, db
def relu_backward(dout, X):
    dX = dout * (X > 0)
    return dX
def maxpool_backward(dout, X, size=2, stride=2):
    N, C, H, W = X.shape
    out_h, out_w = dout.shape[2], dout.shape[3]
    dX = np.zeros_like(X)

    for n in range(N):
        for c in range(C):
            for i in range(out_h):
                for j in range(out_w):
                    h_start = i * stride
                    w_start = j * stride
                    window = X[n, c, h_start:h_start+size, w_start:w_start+size]
                    max_val = np.max(window)
                    for ii in range(size):
                        for jj in range(size):
                            if window[ii, jj] == max_val:
                                dX[n, c, h_start+ii, w_start+jj] += dout[n, c, i, j]
                                break
                        else:
                            continue
                        break
    return dX


In [8]:
def conv2d_backward(dout, X, W, b, stride=1, padding=0):
    N, C, H, W_in = X.shape
    F, _, HH, WW = W.shape
    out_h, out_w = dout.shape[2], dout.shape[3]

    X_pad = np.pad(X, ((0,0),(0,0),(padding,padding),(padding,padding)), mode='constant')
    dX_pad = np.zeros_like(X_pad)
    dW = np.zeros_like(W)
    db = np.zeros_like(b)

    for n in range(N):
        for f in range(F):
            for i in range(out_h):
                for j in range(out_w):
                    h_start = i * stride
                    w_start = j * stride
                    window = X_pad[n, :, h_start:h_start+HH, w_start:w_start+WW]

                    dW[f] += window * dout[n, f, i, j]

                    dX_pad[n, :, h_start:h_start+HH, w_start:w_start+WW] += W[f] * dout[n, f, i, j]

            db[f] += np.sum(dout[n, f])

    if padding > 0:
        dX = dX_pad[:, :, padding:-padding, padding:-padding]
    else:
        dX = dX_pad

    return dX, dW, db

In [9]:
def sgd_update(params, grads, learning_rate=0.01):
    for key in params.keys():
        params[key] -= learning_rate * grads[key]


In [ ]:
learning_rate = 0.01
batch_size = 64
num_epochs = 10

params = initialize_params()

for epoch in range(num_epochs):
    idx = np.random.permutation(len(train_images))
    train_images_shuffled = train_images[idx]
    train_labels_shuffled = train_labels[idx]

    epoch_loss = 0.0
    num_batches = len(train_images) // batch_size

    for i in range(num_batches):

        X_batch = train_images_shuffled[i*batch_size:(i+1)*batch_size]
        y_batch = train_labels_shuffled[i*batch_size:(i+1)*batch_size]

        # Forward
        X1 = conv2d_forward(X_batch, params['W1'], params['b1'], stride=1, padding=1)
        A1 = relu_forward(X1)
        P1 = maxpool_forward(A1, size=2, stride=2)

        X2 = conv2d_forward(P1, params['W2'], params['b2'], stride=1, padding=2)
        A2 = relu_forward(X2)
        P2 = maxpool_forward(A2, size=2, stride=2)

        X3 = conv2d_forward(P2, params['W3'], params['b3'], stride=1, padding=1)
        A3 = relu_forward(X3)

        X4 = conv2d_forward(A3, params['W4'], params['b4'], stride=1, padding=1)
        A4 = relu_forward(X4)

        X5 = conv2d_forward(A4, params['W5'], params['b5'], stride=1, padding=1)
        A5 = relu_forward(X5)
        P5 = maxpool_forward(A5, size=2, stride=2)

        N = P5.shape[0]
        flatten = P5.reshape(N, -1)

        FC1 = fc_forward(flatten, params['W6'], params['b6'])
        A6 = relu_forward(FC1)

        FC2 = fc_forward(A6, params['W7'], params['b7'])
        A7 = relu_forward(FC2)

        FC3 = fc_forward(A7, params['W8'], params['b8'])

        # Loss and backward
        loss, dFC3 = softmax_crossentropy_loss(FC3, y_batch)
        epoch_loss += loss

        grads = {}

        dA7, grads['W8'], grads['b8'] = fc_backward(dFC3, A7, params['W8'], params['b8'])
        dFC2 = relu_backward(dA7, FC2)
        dA6, grads['W7'], grads['b7'] = fc_backward(dFC2, A6, params['W7'], params['b7'])
        dFC1 = relu_backward(dA6, FC1)
        dflatten, grads['W6'], grads['b6'] = fc_backward(dFC1, flatten, params['W6'], params['b6'])

        dP5 = dflatten.reshape(P5.shape)
        dA5 = maxpool_backward(dP5, A5, size=2, stride=2)

        dX5 = relu_backward(dA5, X5)
        dA4, grads['W5'], grads['b5'] = conv2d_backward(dX5, A4, params['W5'], params['b5'], stride=1, padding=1)

        dX4 = relu_backward(dA4, X4)
        dA3, grads['W4'], grads['b4'] = conv2d_backward(dX4, A3, params['W4'], params['b4'], stride=1, padding=1)

        dX3 = relu_backward(dA3, X3)
        dP2, grads['W3'], grads['b3'] = conv2d_backward(dX3, P2, params['W3'], params['b3'], stride=1, padding=1)

        dA2 = maxpool_backward(dP2, A2, size=2, stride=2)

        dX2 = relu_backward(dA2, X2)
        dP1, grads['W2'], grads['b2'] = conv2d_backward(dX2, P1, params['W2'], params['b2'], stride=1, padding=2)

        dA1 = maxpool_backward(dP1, A1, size=2, stride=2)

        dX1 = relu_backward(dA1, X1)
        dX, grads['W1'], grads['b1'] = conv2d_backward(dX1, X_batch, params['W1'], params['b1'], stride=1, padding=1)

        sgd_update(params, grads, learning_rate=learning_rate)

        if i % 10 == 0:
            print(f'Epoch {epoch+1}/{num_epochs} - Batch {i+1}/{num_batches}')

    print(f'Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss/num_batches:.4f}')


In [ ]:
import numpy as np

# ファイルから読み込む
loaded = np.load("saved_params.npz")

# params 辞書を復元
params = {k: loaded[k] for k in loaded.files}

print("✅ パラメータを復元しました！")


In [16]:
import numpy as np

# 学習パラメータを保存
params_np = {k: v for k, v in params.items()}  # NumPy版なのでそのまま
np.savez("saved_params.npz", **params_np)
print("✅ パラメータ保存完了")


✅ パラメータ保存完了


In [10]:
loaded = np.load("saved_params.npz", allow_pickle=False)
print(loaded.files)  # 含まれるキーだけ確認


['W1', 'b1', 'W2', 'b2', 'W3', 'b3', 'W4', 'b4', 'W5', 'b5', 'W6', 'b6', 'W7', 'b7', 'W8', 'b8']


In [11]:
loaded = np.load("saved_params.npz", mmap_mode='r')
params = {k: loaded[k] for k in loaded.files}
